<a href="https://colab.research.google.com/github/VSusko/IA/blob/main/Apresentacao-disciplina/Agentes-implementacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Implementação - Data Center**

A seguir, será apresentada a implementação da funcionalidade de uma *smart house* para um Data Center. O objetivo deste código é, a partir de uma simulação, comparar o desempenho da aplicação da média móvel e da média simples para determinar a compra de suprimentos para o Data Center. 

O ambiente possui as seguintes características:
* A bateria do Data Center tem uma capacidade máxima de 500KWh
* Todos os dias uma certa quantidade aleatória de cargas é consumida no Data Center, e este valor flutua aleatoriamente entre 200 e 400KWh
* Todos os dias o preço do KWh varia entre `R$0,50` e `R$2,00`
* No primeiro dia, o Data Center possui uma carga inicial de 300KWh
* Se a bateria atingir 0 KWh, o Data Center desliga, ou seja, a simulação é interrompida

Assim sendo, o agente da smart house deve decidir diariamente comprar ou não mais cargas para suprir o Data Center dadas as condições:
* O agente comprará cargas se o preço atual do KWh é menor do que a média (simples ou móvel) dos últimos dias. No caso da média móvel, serão considerados apenas os últimos 5 dias, enquanto a média simples considera a média de todos os dias já decorridos
* O agente também comprará cargas se o estoque do Data Center atingir um valor menor ou igual a 200Kwh
* Quando o agente opta por comprar cargas, ele comprará até 400Kwh. A quantidade comprada dependerá do estoque atual do Data Center, sendo que se ele possuir valor maior que 100KWh, como a capacidade máxima da bateria é de 500Kwh, então será comprada apenas a diferença para completar o estoque da bateria
* Caso nenhuma das condições de compra seja satisfeita, o agente não comprará nada no dia



# Importação das bibliotecas necessárias para executar o código


In [ ]:
from random import *
import matplotlib.pyplot as plt
import numpy as np

# Definição das constantes do código

Seguem abaixo as constantes utilizadas no código: o número de dias em cada simulação; o número total de simulações e a diretiva para impressão de valores como média, estoque, quantidade de carga comprada, etc.

In [ ]:
# Definição das constantes
QTD_DIAS       = 20    # Quantidade de dias simulados
NUM_SIMULACOES = 2000  # Número total de simulações
MODO_IMPRESSAO = False # Define se serão mostrados os valores no terminal

# Funções de geração dos números aleatórios

Duas funções foram criadas para gerar números aleatórios. A função *gerar_consumos()* cria um vetor com números aleatórios no intervalo de 200 a 400, para representar os consumos diários do Data Center. Já a função *gerar_precos()* fornece um vetor com números aleatórios no intervalo de 0,5 a 2, para representar as flutuações diárias do preço do KWh. Vale notar que existe uma condição que faz com que o preço do KWh atinja 20 reais, fato que extrapola o limite do intervalo de preços mencionado. Este evento foi adicionado para que possamos observar o comportamento das médias na presença de ruídos, que são valores muito diferentes do padrão esperado. Desse modo, existe 1 chance em 25 de que o valor 20 apareça. Como a média móvel fornece a média dos últimos 5 dias, é esperado que o ruído influencie muito dentro da janela de tempo, mas por poucos dias, enquanto na média simples, os ruídos podem ser mais ou menos influentes, a depender do número de dias. 

In [ ]:
# Função que gera um vetor de numeros entre 200 e 400 para o consumo diario
def gerar_consumos():
  # Vetor que armazenará os valores para a simulação dos consumos diários
  consumos = []
  # São gerados 20 valores entre 200KWh e 400KWh
  for i in range(QTD_DIAS): 
    consumos.append(randint(200, 400))
  return consumos

# Função que gera um vetor de numeros entre 0.5 e 2 para o preço do Kwh
def gerar_precos():
  # Vetor que armazenará os valores para a simulação do preço do KWh
  precos = []
  # São gerados 20 valores entre R$0.50 e R$2.00, mas existe uma chance de 10% de aparecer um preço de 7, ou seja, muito fora do padrão
  for i in range(QTD_DIAS): 
    if(randint(1, 25) == 1):
      precos.append(20)
    else:
      precos.append(0.5 + 1.5 * random())
      
  return precos

# Definição do ambiente

Vamos armazenar no ambiente as seguintes informações:

*   Informação do dia
*   Preço do produto em cada um dos dias da série
*   Histórico de preços
*   Histórico de estoque
*   Histórico com a quantidade de produtos comprados

In [ ]:
# Definicao da classe ambiente
class Ambiente():

  def __init__(self):
    # Ambiente explorado pelo agente de compra de papel higienico
    self.num_dias=0                       # Valor do numero de dias
    self.estoque=300                      # Valor do estoque inicial
    self.historico_preco=[1.5]            # Lista do estoque de preços
    self.historico_estoque=[self.estoque] # Lista do historico de estoques
    self.historico_qtde_comprados=[0]     # Lista da quantidade de produtos comprados
    self.precos_aleatorios = gerar_precos()     # Vetor do preco de cada dia
    self.consumos_aleatorios = gerar_consumos() # Vetor do consumo de cada dia

  # Função que retorna o preço atual do kwh
  def percebe_preco_atual(self):
    return self.historico_preco[len(self.historico_preco)-1]
  
  # Função que retorna o estoque atual
  def percebe_estoque(self):
    return self.historico_estoque[len(self.historico_estoque)-1]    

  # Função que simula o ambiente
  def run(self, dic_acoes, iteracao):
    '''Realizar alteracoes no ambiente: 
       Definir, aleatoriamente, uma quantidade de produtos consumidos
       Atualizar o historico do preco atual e do estoque.
       Essas informacoes serao utilizadas pelo agente para decidir a compra ou nao de produtos
    '''
    # Consumo realizado (valores gerados aleatoriamente)
    if MODO_IMPRESSAO:
      print(f"Estoque atual: {self.historico_estoque[len(self.historico_estoque)-1]}")
      print(f"Consumo realizado no dia: {self.consumos_aleatorios[iteracao]}") # novo valor da quantidade consumida
    estoque_atual = self.historico_estoque[len(self.historico_estoque)-1] - self.consumos_aleatorios[iteracao] + dic_acoes["comprar"]
    if MODO_IMPRESSAO:
      print(f"Estoque final após consumo e compra: {estoque_atual}")
    
    self.historico_estoque.append(estoque_atual)               # Adicionando o estoque atual no histórico
    self.historico_qtde_comprados.append(dic_acoes["comprar"]) # Adicionando quantidade de carga comprada no histórico

    # Informando valor do produto no periodo (Atualizacao para o proximo dia)
    self.historico_preco.append(self.precos_aleatorios[iteracao]) # novo valor do produto, obtido pelo vetor de precos.
    if MODO_IMPRESSAO:
      print(f"Novo valor do preço: {self.precos_aleatorios[iteracao]}")

# Definição do agente

O agente vai armazenar informações sobre:
*   Número de dias já simulados
*   Ambiente (Composição de classes)
*   Informação sobre o estoque atual de cargas
*   Informação sobre o total de dinheiro gasto na simulação
*   Informação do preço atual do KWh
*   Valor da média (simples ou móvel) do preço do KWh

In [ ]:
# Definição da classe agente
class Agente():
  
  def __init__(self, ambiente):
    self.num_dias = 1                                               # Definição do número de dias
    self.ambiente= ambiente                                         # Copia do objeto ambiente
    self.estoque= ambiente.percebe_estoque()                        # Definição do estoque inicial
    self.total_gasto = 0                                            # Variável para armazenar a quantidade total gasta
    self.preco_atual = self.media = ambiente.percebe_preco_atual()  # Variável para o preço atual do Kwh

  # Função que executa o agente, decidindo ou não comprar mais cargas 
  def executa_agente(self, media_movel):
    
    # Loop principal
    for i in range(QTD_DIAS): 
      # O agente percebe o estado do ambiente
      if MODO_IMPRESSAO:
        print(f"Dia: {i+1}")

      self.estoque= self.ambiente.percebe_estoque()         # Obtenção do novo estoque
      self.preco_atual= self.ambiente.percebe_preco_atual() # Obtenção do novo preço
      
      '''
        Controlador do agente:
        - Define a regra para compra de produtos:
          Se a o preço estiver menor que a média, completar a bateria e comprar o consumo do dia
          
          Se o estoque estiver em caso muito crítico (<=100), comprar o consumo do dia + 200 de carga para a bateria
          
          Se o estoque estiver em caso crítico (<=200), comprar metade do consumo do dia + 100 de carga para a bateria

          Se o estoque estiver em caso moderado (<300), comprar um quarto do consumo do dia + 100 de carga para a bateria
          
          Caso contrário, não comprar nada
      '''
      if self.preco_atual < self.media:
        compra = self.ambiente.consumos_aleatorios[i] + 500 - self.estoque  
      elif self.estoque <= 100:
        compra = self.ambiente.consumos_aleatorios[i] + 200
      elif self.estoque <= 200:
        compra = (self.ambiente.consumos_aleatorios[i] / 2) + 100
      elif self.estoque < 300:
        compra = (self.ambiente.consumos_aleatorios[i] / 4) + 100
      else:
        compra = 0
      
      if MODO_IMPRESSAO:      
        print(f"Compra = {compra}")
      
      # Fim do controlador
      self.total_gasto += self.preco_atual*compra
      # O agente aplica modificacoes ao ambiente)
      self.ambiente.run({"comprar": compra}, i)
      # Aumentando o contador de dias
      self.num_dias+=1
      # Cálculo das médias: o primeiro caso representa a média móvel, e o segundo a média simples
      if media_movel and self.num_dias > 5:
        self.media = (self.media*(5) + self.preco_atual - self.ambiente.historico_preco[(self.num_dias - 1) - 5])/5
      else:
        self.media = (self.media*(self.num_dias-1) + self.preco_atual)/self.num_dias
      
      if MODO_IMPRESSAO:      
        print(f"Media atual = {self.media}")
        print(f"\n")
      
      # Se o estoque zerar, a simulação termina
      if self.ambiente.percebe_estoque() <= 0 and i > 1:
        return self.num_dias
      
    return self.num_dias

# Definção da classe para impressão dos resultados


In [ ]:
# Classe para a impressão dos gráficos
class Imprime:
  @staticmethod
  def imprime_resultado(agente):
    historico_dias = np.linspace(0, agente.num_dias, agente.num_dias)

    # Primeira impressão: historico do preco
    fig = plt.figure(figsize=(16,8))
    spec = fig.add_gridspec(1,3)

    axis1 = fig.add_subplot(spec[0,0])
    axis2 = fig.add_subplot(spec[0,1])
    axis3 = fig.add_subplot(spec[0,2])

    axis1.plot(historico_dias, agente.ambiente.historico_preco, 'b-', label='Historico (preço)')
    axis1.legend()

    # Segunda impressão: historico qtde itens comprados
    axis2.vlines(historico_dias, ymin=0, ymax=agente.ambiente.historico_qtde_comprados)
    axis2.plot(historico_dias, agente.ambiente.historico_qtde_comprados, "g-")
    axis2.set_ylim(0, 800)

    # Terceira impressão: historico do estoque
    axis3.plot(historico_dias, agente.ambiente.historico_estoque, 'r-', label='Historico (estoque)')
    axis3.legend()
    plt.show()

# Aspectos Teóricos

Uma vez que o objetivo é comparar as duas médias, a métrica de comparação será o dinheiro total gasto para cada cenário. Dessa forma, o código será executado várias vezes e será observado em qual das médias o Data Center teve menos despesas

Vale mencionar a diferença entre média simples e média móvel. A média simples constitui a razão entre a soma de todos os preços do KWh e o número de dias decorridos durante a simulação. Por outro lado, a média móvel corresponde à média dos preços do KWh apenas dos últimos 5 dias. Dessa forma, para cada dia depois do quinto dia de simulação, a média do preço é calculada com base nos preços dos 5 dias anteriores. Os casos em que a simulação dura até 5 dias não são considerados na análise, uma vez que a média móvel possui uma janela de 5 dias, e portanto, as duas médias terão valores equivalentes nesse período, não sendo passível compará-las. 

Uma vez explicitadas tais diferenças, é possível inferir que a média móvel tende a refletir melhor os preços recentes do KWh, pois considera apenas os últimos 5 dias da simulação. Isso pode tornar o agente de compra mais reativo a tendências de curto prazo, favorecendo decisões mais adaptadas ao momento atual. Por outro lado, a média simples incorpora todo o histórico da simulação, o que pode diluir a influência de alterações recentes nos preços — resultando, em alguns casos, na decisão de não comprar suprimentos quando os preços atuais já subiram.

# Execução do programa

Por um determinado número de dias (iterações), veremos como o agente reagirá conforme o cálculo da média. Para centralizar a análise do agente nas médias, o ambiente simulado possui as mesmas caractéristicas, por isso, são usados os vetores das funções de geração de números aleatórios anteriormente apresentados para armazenar os valores do preço do KWh e do consumo diário. Dessa forma, os dois agentes estarão condicionados ao mesmo ambiente. 

In [ ]:
# Agente da média simples
ambiente = Ambiente()
smart_house_media_simples = Agente(ambiente)
total_dias_simulados_simples = smart_house_media_simples.executa_agente(False)
Imprime.imprime_resultado(smart_house_media_simples)

# Reset do ambiente
ambiente.num_dias=0                       
ambiente.estoque=300                      
ambiente.historico_preco=[1.5]            
ambiente.historico_estoque=[ambiente.estoque] 
ambiente.historico_qtde_comprados=[0] 

# Agente da média móvel
smart_house_media_movel = Agente(ambiente)
total_dias_simulados_movel = smart_house_media_movel.executa_agente(True)
Imprime.imprime_resultado(smart_house_media_movel)

print(f"Total de dias simulados na média simples: {total_dias_simulados_simples}\n")
print(f"Total gasto na média simples: {smart_house_media_simples.total_gasto}\n")
print(f"Total de dias simulados na média móvel: {total_dias_simulados_movel}\n")
print(f"Total gasto na média móvel: {smart_house_media_movel.total_gasto}")

Para a obtenção dos resultados, utilizando o código mostrado no tópico acima esta tarefa pode ser feita manualmente, observando elementos do gráfico e registrando quando uma média teve performance melhor que outra. Os gráficos mostram resultados como: 

![alt text](Figure_1.png)
![alt text](Figure_2.png)

Entretanto, como podemos perceber, essa simulação não compreende uma simulação válida visto que durou apenas 3 dias e não podemos comparar as médias. Assim sendo, o fato de que podemos ter simulações descartáveis e que queremos obter uma análise estatística mais precisa, ou seja, com um número grande de simulações para obter resultados mais coerentes, uma nova implementação da última parte do programa foi realizada, apenas para que seja possível visualizar quantas vezes uma média foi superior à outra. 

O novo código não mostrará graficamente os resultados, porém será possível realizar as simulações de forma muito mais ágil. Além disso, serão contabilizadas apenas as simulações em que o número de dias ultrapassou 5 e as duas médias mantiveram o Data Center ativo durante o máximo de dias preestabelecido. Assim sendo, os testes serão divididos da seguinte forma:
*   2000 simulações com um periódo de 20 dias em cada
*   2000 simulações com um periódo de 50 dias em cada
*   2000 simulações com um periódo de 100 dias em cada
*   2000 simulações com um periódo de 150 dias em cada
*   2000 simulações com um periódo de 200 dias em cada
*   2000 simulações com um periódo de 300 dias em cada

Desse modo, o programa contará quantas vezes qual média produziu um gasto menor, bem como os casos de empate. 

In [ ]:
metrica_media_simples_total_gasto = 0 # Variável que contabiliza quantas vezes a média simples produziu gastos menores
metrica_media_movel_total_gasto = 0   # Variável que contabiliza quantas vezes a média móvel produziu gastos menores
empate_total_gasto = 0 # Variável que contabiliza empate no dinheiro total gasto nas duas médias
simulacoes = 0 # Número de simulacoes totais
while True:
  # Agente da média simples
  ambiente = Ambiente()
  smart_house_media_simples = Agente(ambiente)
  total_dias_simulados_simples = smart_house_media_simples.executa_agente(False)

  # Se o numero de dias simulado for menor ou igual a 5, recomece
  if total_dias_simulados_simples < QTD_DIAS: 
    continue
  
  # Reset do ambiente
  ambiente.num_dias=0                       
  ambiente.estoque=300                      
  ambiente.historico_preco=[1.5]            
  ambiente.historico_estoque=[ambiente.estoque] 
  ambiente.historico_qtde_comprados=[0]     
  
  # Agente da média móvel
  smart_house_media_movel = Agente(ambiente)
  total_dias_simulados_movel = smart_house_media_movel.executa_agente(True)
  
  # Se o numero de dias for menor ou igual a 5, recomece
  if total_dias_simulados_movel < QTD_DIAS:
    continue
  
  if MODO_IMPRESSAO:
    print(f"Total de dias simulados: {QTD_DIAS}\n")
    print(f"Total gasto (media simples): {smart_house_media_simples.total_gasto}\n")
    print(f"Total gasto (media movel): {smart_house_media_movel.total_gasto}")
  
  ''' Se a media simples produziu menos gasto, o contador aumenta em 1 
      Se a media movel produziu menos gasto, o contador aumenta em 1
      Em caso de empate, nenhumas das duas aumenta e ele é contabilizado
  '''
  if smart_house_media_simples.total_gasto < smart_house_media_movel.total_gasto:
     metrica_media_simples_total_gasto += 1
  elif smart_house_media_simples.total_gasto > smart_house_media_movel.total_gasto:
     metrica_media_movel_total_gasto += 1
  else:
    empate_total_gasto += 1
  
  # Aumento do numero de simulacoes
  simulacoes += 1
  
  if simulacoes == NUM_SIMULACOES:
    break

print(f"Número de vezes em que a média simples gerou menos custo: {metrica_media_simples_total_gasto}\n")
print(f"Número de vezes em que a média móvel gerou menos custo: {metrica_media_movel_total_gasto}\n")
print(f"Número de empates do custo: {empate_total_gasto}\n")

print(f"A média simples faz com que o Data Center tenha menos despesas em {metrica_media_simples_total_gasto/NUM_SIMULACOES*100:.2f}% dos casos\n")
print(f"A média móvel faz com que o Data Center tenha menos despesas em {metrica_media_movel_total_gasto/NUM_SIMULACOES*100:.2f}% dos casos\n")
print(f"As duas médias são iguais quanto às despesas do Data Center em {empate_total_gasto/NUM_SIMULACOES*100:.2f}% dos casos\n")

# Resultados

Para o cenário de 2000 simulações com um periódo de `20` dias em cada, obtemos os seguintes números:
*   Número de vezes em que a média simples gerou menos custo: 540
*   Número de vezes em que a média móvel gerou menos custo: 937
*   Número de empates do custo: 523

Logo, as estatísticas deste cenário são:
*   A média simples faz com que o Data Center tenha menos despesas em 27.00% dos casos
*   A média móvel faz com que o Data Center tenha menos despesas em 46.85% dos casos
*   As duas médias são iguais quanto às despesas do Data Center em 26.15% dos casos

Para o cenário de 2000 simulações com um periódo de `50` dias em cada, obtemos os seguintes números:
*   Número de vezes em que a média simples gerou menos custo: 932
*   Número de vezes em que a média móvel gerou menos custo: 1053
*   Número de empates do custo: 15

Logo, as estatísticas deste cenário são:
*   A média simples faz com que o Data Center tenha menos despesas em 46.60% dos casos
*   A média móvel faz com que o Data Center tenha menos despesas em 52.65% dos casos
*   As duas médias são iguais quanto às despesas do Data Center em 0.75% dos casos

Para o cenário de 2000 simulações com um periódo de `100` dias em cada, obtemos os seguintes números:
*   Número de vezes em que a média simples gerou menos custo: 1164
*   Número de vezes em que a média móvel gerou menos custo: 836
*   Número de empates do custo: 0

Logo, as estatísticas deste cenário são:
*   A média simples faz com que o Data Center tenha menos despesas em 58.20% dos casos
*   A média móvel faz com que o Data Center tenha menos despesas em 41.80% dos casos
*   As duas médias são iguais quanto às despesas do Data Center em 0.00% dos casos

Para o cenário de 2000 simulações com um periódo de `150` dias em cada, obtemos os seguintes números:
*   Número de vezes em que a média simples gerou menos custo: 1285
*   Número de vezes em que a média móvel gerou menos custo: 715
*   Número de empates do custo: 0

Logo, as estatísticas deste cenário são:
*   A média simples faz com que o Data Center tenha menos despesas em 64.25% dos casos
*   A média móvel faz com que o Data Center tenha menos despesas em 35.75% dos casos
*   As duas médias são iguais quanto às despesas do Data Center em 0.00% dos casos

Para o cenário de 2000 simulações com um periódo de `200` dias em cada, obtemos os seguintes números:
*   Número de vezes em que a média simples gerou menos custo: 1389
*   Número de vezes em que a média móvel gerou menos custo: 611
*   Número de empates do custo: 0

Logo, as estatísticas deste cenário são:
*   A média simples faz com que o Data Center tenha menos despesas em 69.45% dos casos
*   A média móvel faz com que o Data Center tenha menos despesas em 30.55% dos casos
*   As duas médias são iguais quanto às despesas do Data Center em 0.00% dos casos

Para o cenário de 2000 simulações com um periódo de `300` dias em cada, obtemos os seguintes números:
*   Número de vezes em que a média simples gerou menos custo: 1517
*   Número de vezes em que a média móvel gerou menos custo: 483
*   Número de empates do custo: 0

Logo, as estatísticas deste cenário são:
*   A média simples faz com que o Data Center tenha menos despesas em 75.85% dos casos
*   A média móvel faz com que o Data Center tenha menos despesas em 24.15% dos casos
*   As duas médias são iguais quanto às despesas do Data Center em 0.00% dos casos


# Conclusões

Percebemos que nos cenários de 20 dias e 50 dias, a média móvel teve desempenho superior mais vezes. Porém, em todos os cenários seguintes, conforme o número de dias por simulação aumentava, a média simples apresentou uma performance cada vez melhor.  Um possível motivo para este comportamento é que quanto mais dias estão presentes em cada simulação, mais o preço dos ruídos é diluído na média simples, enquanto a média móvel é necessariamente afetada por pelo menos 5 dias a cada ruído. Por isso, embora a média móvel reaja mais rapidamente às mudanças de preço, ela é mais vulnerável a picos extremos, o que pode comprometer o desempenho em cenários de longo prazo.